In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Brain Tumor Segmentation — Hackathon Fast Pipeline

Model: U-Net with EfficientNetB0 pretrained encoder
Modalities: FLAIR + T1ce
Strategy:
- 2.5D slice segmentation
- tumor-slice filtering
- memory-safe generator
- Dice loss
- RLE submission


In [ ]:
import os
import glob
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence

In [ ]:
import nibabel as nib

def load_nii(file_path):
    return nib.load(file_path).get_fdata()

In [ ]:
def process_patient(patient_path):
    import glob
    # Flair & T1ce loading
    flair_files = glob.glob(os.path.join(patient_path,"*_flair.nii","*.nii")) or glob.glob(os.path.join(patient_path,"*_flair.nii"))
    t1ce_files = glob.glob(os.path.join(patient_path,"*_t1ce.nii","*.nii")) or glob.glob(os.path.join(patient_path,"*_t1ce.nii"))
    
    if not flair_files or not t1ce_files: return None, None
    
    f = load_nii(flair_files[0])
    t = load_nii(t1ce_files[0])

    def min_max_normalize(img):
        img = np.nan_to_num(img)
        low, high = np.percentile(img, [1, 99])
        img = np.clip(img, low, high)
        if (high - low) > 0:
            return (img - low) / (high - low)
        return img

    slice_idx = f.shape[2] // 2
    f_slice = min_max_normalize(f[:,:,slice_idx])
    t_slice = min_max_normalize(t[:,:,slice_idx])

    img_slice = np.stack([f_slice, t_slice], axis=-1).astype(np.float32)
    img_slice = tf.image.resize(img_slice, (256,256))

    # Mask processing
    mask_files = glob.glob(os.path.join(patient_path,"*_seg.nii","*.nii")) or glob.glob(os.path.join(patient_path,"*_seg.nii"))
    if not mask_files: return img_slice, None
    
    m = load_nii(mask_files[0])
    m_new = np.zeros_like(m)
    m_new[m==1] = 1 # NCR
    m_new[m==2] = 2 # ED
    m_new[m==4] = 3 # ET
    
    mask_slice = tf.image.resize(m_new[:,:,slice_idx][..., np.newaxis], (256,256), method="nearest")
    mask_cat = tf.keras.utils.to_categorical(mask_slice, num_classes=4)
    return img_slice, tf.cast(mask_cat, tf.float32)

In [ ]:
class BrainGenerator(Sequence):
    def __init__(self, paths, batch_size=4, shuffle=True):
        self.paths = paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_paths = self.paths[idx*self.batch_size : (idx+1)*self.batch_size]
        imgs, masks = [], []
        for p in batch_paths:
            img, mask = process_patient(p)
            if img is None or mask is None:
                continue
            imgs.append(img)
            masks.append(mask)
        if len(imgs)==0:
            return self.__getitem__((idx+1) % self.__len__())
        return np.array(imgs), np.array(masks)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.paths)

In [ ]:
TRAIN_DIR = "/kaggle/input/instant-odc-ai-hackathon/Train"
all_patients = glob.glob(os.path.join(TRAIN_DIR,"*"))
random.shuffle(all_patients)
split = int(len(all_patients)*0.85)
train_paths = all_patients[:split]
val_paths = all_patients[split:]

train_gen = BrainGenerator(train_paths, batch_size=4)
val_gen = BrainGenerator(val_paths, batch_size=4)

print("Train patients:", len(train_paths), "Validation patients:", len(val_paths))


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

# Input: 256x256x2
inp = layers.Input(shape=(256,256,2))

# Convert 2 channels → 3 channels for pretrained MobileNetV2
x = layers.Concatenate()([inp, inp[:,:,:,0:1]])  # shape = (256,256,3)

# Pretrained MobileNetV2 encoder
encoder = MobileNetV2(include_top=False, weights="imagenet", input_tensor=x)

# Extract feature maps from encoder
x = encoder.output  # shape ~ (8,8,1280) or (16,16,1280) depending on MobileNetV2

# Decoder: Upsampling blocks
x = layers.Conv2D(512, 3, padding="same", activation="relu")(x)
x = layers.UpSampling2D(2)(x)  # double size
x = layers.Conv2D(256, 3, padding="same", activation="relu")(x)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(32, 3, padding="same", activation="relu")(x)
x = layers.UpSampling2D(2)(x)

# Final output: 256x256x4
out = layers.Conv2D(4, 1, activation="softmax")(x)

model = models.Model(inputs=inp, outputs=out)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true[..., 1:]) 
    y_pred_f = K.flatten(y_pred[..., 1:])
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def combined_loss(y_true, y_pred):
    ce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    dl = dice_loss(y_true, y_pred)
    return ce + dl 

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
    loss=combined_loss, 
    metrics=[dice_coef, 'accuracy']
)

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15 
)

In [ ]:

model.save("brats_segmentation_model.h5")
print("Model saved successfully as brats_segmentation_model.h5")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# المسار اللي نسختيه
model_path = '/kaggle/working/brats_segmentation_model.h5'

# تحميل الموديل
try:
    model = load_model(model_path)
    print("Model Saved Succesfully")
except Exception as e:
    print(f" There is a problem {e}")

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import nibabel as nib
import glob
import tensorflow as tf

def rle_encode_final(mask_3d):
    pixels = mask_3d.T.flatten() 
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    rle_string = ' '.join(str(x) for x in runs)
    return rle_string if rle_string != "" else "1 1"

def min_max_normalize(img):
    img = np.nan_to_num(img)
    low, high = np.percentile(img, [1, 99])
    img = np.clip(img, low, high)
    if (high - low) > 0:
        return (img - low) / (high - low)
    return img

def predict_patient_ultimate_v2(patient_path):
    path_flair = glob.glob(os.path.join(patient_path, '*_flair.nii'))[0]
    path_t1ce  = glob.glob(os.path.join(patient_path, '*_t1ce.nii'))[0]
    
    v_flair = nib.load(path_flair).get_fdata()
    v_t1ce  = nib.load(path_t1ce).get_fdata()
    
    v_flair_norm = min_max_normalize(v_flair)
    v_t1ce_norm  = min_max_normalize(v_t1ce)
    
    final_volume = np.zeros((240, 240, 155, 4), dtype=np.float32)
    
    for slice_idx in range(155):
        
        f_slice = v_flair_norm[..., slice_idx]
        t_slice = v_t1ce_norm[..., slice_idx]
        
        img_input = np.stack([f_slice, t_slice], axis=-1).astype(np.float32)
    
        img_input = tf.image.resize(img_input, (256, 256)).numpy() 
        
        pred = model.predict(np.expand_dims(img_input, axis=0), verbose=0)[0]
        
        pred_res = cv2.resize(pred, (240, 240), interpolation=cv2.INTER_NEAREST)
        final_volume[:, :, slice_idx, :] = pred_res
        
    return final_volume

submission_list = []
for i, patient_path in enumerate(test_folders):
    p_id = os.path.basename(patient_path)
    if "BraTS2021" not in p_id: continue
    
    print(f"⌛ Processing with Original Normalization: {p_id}")
    full_pred = predict_patient_ultimate_v2(patient_path)
    
    for suffix, ch_idx in {"1": 1, "2": 2, "4": 3}.items():
        mask_binary = (full_pred[..., ch_idx] > 0.5).astype(np.uint8)
        submission_list.append({"id": f"{p_id}_{suffix}", "rle": rle_encode_final(mask_binary)})

df_final = pd.DataFrame(submission_list)
sample = pd.read_csv("/kaggle/input/instant-odc-ai-hackathon/sample_submission.csv")
final_sub = pd.merge(sample[['id']], df_final, on='id', how='left').fillna("1 1")
final_sub = final_sub.drop_duplicates(subset=['id'], keep='first')
final_sub.to_csv("SUBMISSION_WITH_NORM.csv", index=False)